<a href="https://colab.research.google.com/github/soumilbaldota/MNIST/blob/master/fashion__MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline
try:
  %tensorflow_version 1.x
except:
  pass
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline


TensorFlow 1.x selected.


In [3]:
from keras.datasets import fashion_mnist


Using TensorFlow backend.


In [4]:
((X_train,y_train),(X_test,y_test))=fashion_mnist.load_data()

4423680/4422102 [==============================] - 1s 0us/step


In [5]:
X_train=X_train.reshape(60000, 28, 28, 1)
X_train = X_train/255.0
X_test = X_test/255.0

In [6]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)

In [7]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [8]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if logs.get('acc')>=0.998:
      print("\nReached 99.8% accuracy so cancelling training!")
      self.model.stop_training=True
callbacks=myCallback()

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D,BatchNormalization,Dropout,Dense,Flatten,MaxPooling2D

In [10]:
model = Sequential()
model.add(Conv2D(64,(3,3),padding="same",input_shape=(28,28,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding="same",activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Dense(10,activation='softmax'))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1606144   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)              

In [14]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [15]:
history=model.fit(X_train,y_train,shuffle=True,verbose=1,epochs=20,callbacks=[callbacks],validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 10s 205us/step - loss: 0.2833 - acc: 0.8982 - val_loss: 0.2723 - val_acc: 0.9032
Epoch 2/20
48000/48000 [==============================] - 10s 203us/step - loss: 0.2383 - acc: 0.9145 - val_loss: 0.3023 - val_acc: 0.8918
Epoch 3/20
48000/48000 [==============================] - 10s 200us/step - loss: 0.2107 - acc: 0.9232 - val_loss: 0.2511 - val_acc: 0.9104
Epoch 4/20
48000/48000 [==============================] - 10s 201us/step - loss: 0.1918 - acc: 0.9299 - val_loss: 0.2755 - val_acc: 0.9018
Epoch 5/20
48000/48000 [==============================] - 10s 202us/step - loss: 0.1714 - acc: 0.9374 - val_loss: 0.2400 - val_acc: 0.9202
Epoch 6/20
48000/48000 [==============================] - 10s 212us/step - loss: 0.1548 - acc: 0.9429 - val_loss: 0.2520 - val_acc: 0.9149
Epoch 7/20
48000/48000 [==============================] - 10s 199us/step - loss: 0.1391 - acc: 0.9483 - val_loss: 0.

In [16]:
y_test=to_categorical(y_test)
X_test=X_test.reshape(X_test.shape[0],28,28,1)
test_history=model.evaluate(X_test,y_test)

10000/10000 [==============================] - 1s 74us/step


In [17]:
print("Training accuracy: {:.2f}%".format(history.history['acc'][-1]*100))
print("Validation accuracy: {:.2f}%".format(history.history['val_acc'][-1]*100))
print("Test accuracy: {:.2f}%".format(test_history[1]*100))


Training accuracy: 98.66%
Validation accuracy: 92.02%
Test accuracy: 91.11%


In [18]:
model.save('my_mnist.h5')